<a href="https://colab.research.google.com/github/jyjoon001/regex/blob/main/VN_KR_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Korean Language Preprocessing for Visual Novel Translation

한국어 번역문 검수에 유용하게 사용할 수 있는 Python 전처리 스크립트입니다. 


*   Spell Check
 *   띄어쓰기 검사 (PyKoSpacing)
 * 맞춤법 검사 (hanspell-py)

> @tobisawa


## 1. Basic Preprocessing
Colab에서 라이브러리를 동작할 수 있게 설정합니다.

In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-m6ujqhn6
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-m6ujqhn6
     |████████████████████████████████| 394.7MB 43kB/s 
  Created wheel for pykospacing: filename=pykospacing-0.4-cp36-none-any.whl size=2255638 sha256=a0c738ca05b73faf5c301aad2bd55a6b75a811b91eca11ccc13f560bd5225b6f
  Stored in directory: /tmp/pip-ephem-wheel-cache-mb7r7xc8/wheels/4d/45/58/e26cb2b7f6a063d234158c6fd1e5700f6e15b99d67154340ba
Successfully built pykospacing
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-kdgx5u6u
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-kdgx5u6u
  Created wheel for py-hanspell: filename=py_hanspell-1.1-cp36-none-any.whl size=4854 sha256=964dacd6e0e30c595642a6f6c58d18d0ec9e044538101e3d609424c43be01d9f
  Stored in directory: /tmp/pip-ephem-wheel-cache-u324zcgc/wheels/0a/25/d1/e5e96476dbb1c318cc26c992dd493394fe42b0c204b3e65588
Successfully built py-hanspell


## 2. Spell check
띄어쓰기 검사로는 [한국어 띄어쓰기 검사 라이브러리](https://github.com/haven-jeon/PyKoSpacing)를 사용하고,   
맞춤법 검사로는 [한국어 맞춤법 검사 라이브러리](https://github.com/ssut/py-hanspell)와, 토큰 예외 지정 파일을 통하여 번역할 수 있습니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


먼저 전처리된 번역 스크립트를 다운로드합니다.

In [ ]:
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1Ni5m5puOHeJBXCFYH1KgXBgJxg_N-wKs" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1Ni5m5puOHeJBXCFYH1KgXBgJxg_N-wKs" -o text.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0    841      0 --:--:-- --:--:-- --:--:--   839
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 47078  100 47078    0     0  75445      0 --:--:-- --:--:-- --:--:-- 75445


In [ ]:
# data = open('/content/drive/MyDrive/Konosora_KR/co2.txt', 'r', encoding='utf-8')
data = open('/content/text.txt', 'r', encoding='utf-8')
sent = data.readlines()

In [ ]:
for i in range(0, 10):
    print(sent[i])

「역시 여기 있었구나」

「요루쨩」

「슬슬 돌아가지 않으면 저녁시간에 늦어?」

「응……그래도, 앞으로 조금만 더」

「기다려도 헛수고야. 이제 나타나지 않아」

「어? 어떻게 아는 거야?」

「그 뒤로 한 번도 안 날았잖아. \n 활주로를 보러 갔었는데, 잡초만 자라고 쓰이는 흔적도 없었어」

「응……하지만」

「으―……」

「하아……」



다음으로 맞춤법 검사기에서 오류를 일으키는 텍스트 및 고유명사 등을 one-hot encoding한 커스텀 사전을 다운로드하여 적용합니다.

In [ ]:
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1N2YdlZJgnGj1VwuXMC_vhGq1fYLSe9nW" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1N2YdlZJgnGj1VwuXMC_vhGq1fYLSe9nW" -o encode.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   1707      0 --:--:-- --:--:-- --:--:--  1700
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   504  100   504    0     0   1279      0 --:--:-- --:--:-- --:--:--  1279


In [ ]:
lownword_encode = {}
lownword_decode = {}
lownword_data = open('/content/encode.txt', 'r', encoding='utf-8')
lines = lownword_data.readlines()

for line in lines:
    line = line.strip()
    miss_spell = line.split('\t')[0]
    ori_word = line.split('\t')[1]
    lownword_encode[miss_spell] = ori_word
    lownword_decode[ori_word] = miss_spell

In [ ]:
print(lownword_encode)
print(lownword_decode)

{'&': 'wrd0_2', '료코': 'wrd1_2', '마사츠구': 'wrd2_2', '모닝글로리': 'wrd3_2', '모치즈키': 'wrd4_2', '사나에': 'wrd5_2', '소어링': 'wrd6_2', '시구레': 'wrd7_2', '아게하': 'wrd8_2', '아마네': 'wrd9_2', '아오이': 'wrd10_2', '아카리': 'wrd11_2', '에리': 'wrd12_2', '요루': 'wrd13_2', '유카': 'wrd14_2', '카나코': 'wrd15_2', '카자미': 'wrd16_2', '카자토': 'wrd17_2', '카제가우라': 'wrd18_2', '케이후': 'wrd19_2', '코토리': 'wrd20_2', '쿠모이': 'wrd21_2', '토비오카': 'wrd22_2', '하바네': 'wrd23_2', '호타루': 'wrd24_2', '히메기': 'wrd25_2', '히바리': 'wrd26_2'}
{'wrd0_2': '&', 'wrd1_2': '료코', 'wrd2_2': '마사츠구', 'wrd3_2': '모닝글로리', 'wrd4_2': '모치즈키', 'wrd5_2': '사나에', 'wrd6_2': '소어링', 'wrd7_2': '시구레', 'wrd8_2': '아게하', 'wrd9_2': '아마네', 'wrd10_2': '아오이', 'wrd11_2': '아카리', 'wrd12_2': '에리', 'wrd13_2': '요루', 'wrd14_2': '유카', 'wrd15_2': '카나코', 'wrd16_2': '카자미', 'wrd17_2': '카자토', 'wrd18_2': '카제가우라', 'wrd19_2': '케이후', 'wrd20_2': '코토리', 'wrd21_2': '쿠모이', 'wrd22_2': '토비오카', 'wrd23_2': '하바네', 'wrd24_2': '호타루', 'wrd25_2': '히메기', 'wrd26_2': '히바리'}


맞춤법 검사기를 사용하여, 번역된 개별 스크립트와 오류 갯수, 각 형태소별 오류 형태에 대한 결과를 반환합니다.

In [ ]:
from hanspell import spell_checker

def spell_check_text(texts):
    corpus = []
    errors = []
    errorinfo = []
    for sent in texts:
        for lownword in lownword_encode:
            sent = sent.replace(lownword, lownword_encode[lownword])
        spelled_sent = spell_checker.check(sent)
        for lownword in lownword_decode:
            checked_decoded = spelled_sent.checked.replace(lownword, lownword_decode[lownword])
        corpus.append(checked_decoded)
        errors.append(spelled_sent.errors)
        errorinfo.append(spelled_sent.words)
    return (corpus, errors, errorinfo)

In [ ]:
(spell_preprocessed_corpus, error_num, spell_error_info) = spell_check_text(sent)

list에 각 결과를 저장하여 출력할 수 있습니다. <br/>
이 때 개별 스크립트는 string, 오류 갯수는 int, 각 형태소별 오류 형태는 collections library의 OrderedDict 형태로 저장합니다. 


In [ ]:
for i in range(0, 10):
    print(spell_preprocessed_corpus[i])
    print(error_num[i])
    print(spell_error_info[i])

「역시 여기 있었구나」
0
OrderedDict([('「역시', 0), ('여기', 0), ('있었구나」<br>', 0)])
「wrd13_2짱」
1
OrderedDict([('「wrd13_2짱」', 4)])
「슬슬 돌아가지 않으면 저녁시간에 늦어?」
0
OrderedDict([('「슬슬', 0), ('돌아가지', 0), ('않으면', 0), ('저녁시간에', 0), ('늦어?」<br>', 0)])
「응……그래도, 앞으로 조금만 더」
0
OrderedDict([('「응……그래도,', 0), ('앞으로', 0), ('조금만', 0), ('더」<br>', 0)])
「기다려도 헛수고야. 이제 나타나지 않아」
0
OrderedDict([('「기다려도', 0), ('헛수고야.', 0), ('이제', 0), ('나타나지', 0), ('않아」<br>', 0)])
「어? 어떻게 아는 거야?」
0
OrderedDict([('「어?', 0), ('어떻게', 0), ('아는', 0), ('거야?」<br>', 0)])
「그 뒤로 한 번도 안 날았잖아. \n 활주로를 보러 갔었는데, 잡초만 자라고 쓰이는 흔적도 없었어」
0
OrderedDict([('「그', 0), ('뒤로', 0), ('한', 0), ('번도', 0), ('안', 0), ('날았잖아.', 0), ('\\n', 0), ('활주로를', 0), ('보러', 0), ('갔었는데,', 0), ('잡초만', 0), ('자라고', 0), ('쓰이는', 0), ('흔적도', 0), ('없었어」<br>', 0)])
「응……하지만」
0
OrderedDict([('「응……하지만」<br>', 0)])
「으―……」
0
OrderedDict([('「으―……」<br>', 0)])
「하아……」
0
OrderedDict([('「하아……」<br>', 0)])


각 형태소별 오류 형태를 상수 형태로 출력하는데 이가 나타내는 것은 다음과 같습니다. 

| 상수 | 출력 | 해석
|-----|-----|-----|
|0|.PASSED|맞춤법 검사 결과 문제가 없는 단어 또는 구절|
|1|.WRONG_SPELLING|맞춤법에 문제가 있는 단어 또는 구절|
|2|.WRONG_SPACING|띄어쓰기에 문제가 있는 단어 또는 구절|
|3|.AMBIGUOUS|표준어가 의심되는 단어 또는 구절|
|4|.STATISTICAL_CORRECTION|통계적 교정에 따른 단어 또는 구절|

## 3. Export Data
결과를 스프레드시트로 출력하기 위하여 pandas library의 dataframe을 사용하였습니다.

In [ ]:
import pandas as pd

corpus = [sent, spell_preprocessed_corpus,error_num, spell_error_info]
df = pd.DataFrame (corpus,index=['original', 'corrected', 'error count',  'error case'])

In [ ]:
print(df.T)

                             original  ...                                         error case
0                      「역시 여기 있었구나」\n  ...                {'「역시': 0, '여기': 0, '있었구나」<br>': 0}
1                             「요루쨩」\n  ...                                  {'「wrd13_2짱」': 4}
2           「슬슬 돌아가지 않으면 저녁시간에 늦어?」\n  ...  {'「슬슬': 0, '돌아가지': 0, '않으면': 0, '저녁시간에': 0, '늦...
3               「응……그래도, 앞으로 조금만 더」\n  ...   {'「응……그래도,': 0, '앞으로': 0, '조금만': 0, '더」<br>': 0}
4           「기다려도 헛수고야. 이제 나타나지 않아」\n  ...  {'「기다려도': 0, '헛수고야.': 0, '이제': 0, '나타나지': 0, '...
..                                ...  ...                                                ...
771  「자는 동안에 건물이 와르르 무너져 버릴지도 모르겠네」\n  ...  {'「자는': 0, '동안에': 0, '건물이': 0, '와르르': 0, '무너져'...
772                        「히에에에……」\n  ...    {'<span': 0, 'class='violet_text'>「히에에에……」': 0}
773                   「요루쨩, 같이 자자!」\n  ...       {'「wrd13_2짱,': 4, '같이': 0, '자자!': 2, '」': 2}
774                       「어떻게 할까~」\n  ...                  

In [ ]:
df.T.to_excel('output.xlsx')

In [ ]:
from google.colab import files
files.download("output.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>



***
<br/>

PyKoSpacing이 성능이 별로 좋지 않아서 구동 부분은 주석처리하였습니다. 

In [ ]:
# from pykospacing import spacing

In [ ]:
# spaced_corpus = []
# for i in range(0, 10):
#     spaced_corpus.append(lines[i].replace(" ","")) 
# #    spaced_corpus.append(spacing(lines[i]))
#     spaced_corpus[i] = spacing(spaced_corpus[i])

In [ ]:
# for i in range(0, 10):
#     print(spaced_corpus[i])

In [ ]:
# from hanspell import spell_checker

# corrected_corpus = []
# for i in range(0, len(basic_preprocessed_corpus)-1):
#     spelled_sent = spell_checker.check(basic_preprocessed_corpus[i])
#     corrected_corpus.append(spelled_sent.checked)

In [ ]:
# for i in range(0, len(basic_preprocessed_corpus)-1):
#     print(corrected_corpus[i])

사람 이름은 맞춤법 검사가 잘 되려나? 
안녕하세요 저는 민수에요. 
유 민수이고요, 옆은 김 영희예요.
randnm1이 이 짐을 들어줄 수 있어? 
randnm1가 이 짐을 들어줄 수 있어? 
randnm1은 힘이 세구나. 
randnm1은 힘이 세구나. 
randnm1을 좋아하게 되어 버렸어. 
randnm1을 좋아하게 되어 버렸어. 
randnm2가 이 짐을 들어줄 수 있어? 
randnm2가 이 짐을 들어줄 수 있어? 
randnm2는 힘이 세구나. 
randnm2는 힘이 세구나. 
randnm2를 좋아하게 되어 버렸어. 
randnm2를 좋아하게 되어 버렸어. 
저 새는 날 수 있어? 


In [ ]:
# spelled_sent = spell_checker.check("저 새가 하늘로 하는 날이 올까?")
# print(spelled_sent.errors)
# print(spelled_sent.words)

0
OrderedDict([('저', 0), ('새가', 0), ('하늘로', 0), ('하는', 0), ('날이', 0), ('올까?', 0)])


In [ ]:
# type(spelled_sent.words)

collections.OrderedDict

In [ ]:
# !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1RNYpLE-xbMCGtiEHIoNsCmfcyJP3kLYn" > /dev/null
# !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1RNYpLE-xbMCGtiEHIoNsCmfcyJP3kLYn" -o confused_loanwords.txt